In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_excel("C:/Users/Niku/Documents/Semester8/DataFiles/datasetnorm1.xlsx")
data = pd.read_excel("C:/Users/Niku/Documents/Semester8/DataFiles/dataset2.xlsx")

(1994, 36)

In [4]:
Normdata = data
Normdata = np.matrix(Normdata)
for i in range(1, Normdata.shape[1]):
    cur = Normdata[:, i]
    mx = max(cur)*1.0
    mn = min(cur)*1.0
    for j in range(0, len(cur)):
        cur[j] = (cur[j] - mn)/(mx - mn)
    Normdata[:, i] = cur

cols = Normdata.shape[1]

X = Normdata[:,1:cols-1]
y = Normdata[:,cols-1:cols]

In [5]:
X = np.matrix(X)
y = np.matrix(y)

(1994L, 34L)

In [7]:
for i in range(len(y)):
    if y[i] == 0:
        y[i] = -1

In [9]:
def initialiseLMC():
    c = np.zeros(100000)
    V = np.zeros((100000, X_train.shape[1]-1))
    V = np.insert(V, 0, values=np.ones(V.shape[0]), axis=1)
    k = 0
    return V, c, k

In [10]:
def trainLMC(X_train, y_train, V, c, t, m, k):
    #print 'Training Started'
    m = X_train.shape[0]
    n = X_train.shape[1]
    c = np.zeros(100000)
    #V = np.zeros((996, n))
    #V = np.insert(V, 0, values=np.ones(V.shape[0]), axis=1)
    for e in range(t):
        #print e
        for i in range(0, m-1):
            y_pred = np.sign(np.dot(X_train[i], V[k].T))
            if y_pred == y_train[i]:
                c[k+1] = c[k] + 1
            else:
                d = np.asscalar(y_train[i])
                p = V[k] + (d*X_train[i]).T
                V[k+1] = V[k] + (d*X_train[i])
                c[k+1] = 1
                k = k + 1
    #print 'Done training' 
    #print k
    return V, c, k

In [11]:
def predictLMC(X_test, V, c, k):
    y_LMC = np.zeros(X_test.shape[0])
    a = np.zeros(k)
    for j in range(0, X_test.shape[0]):
        for i in range(0, k):
            a[i] = c[i] * np.sign(np.dot(X_test[j], V[i].T))
            s = np.sum(a)
            y_LMC[j] = np.sign(s)
    #print 'Test Data Created'
    return y_LMC

In [12]:
epoch = [1, 2, 3, 4, 5, 10, 15, 20, 25, 50, 75, 100]
count = 0
prec = np.zeros((12,2))
recal = np.zeros((12,2))
f1 = np.zeros((12,2))
acc = np.zeros((12,2))
auc =  np.zeros((12,2))
K = np.zeros((12,2))
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn import metrics
for i in range(0,12):
    t = epoch[i]
    print t
    for l in range(0,2):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)
        X_train = np.insert(X_train, 0, values=np.ones(X_train.shape[0]), axis=1)
        X_test = np.insert(X_test, 0, values=np.ones(X_test.shape[0]), axis=1)
        V, c, k= initialiseLMC()
        V, c, k = trainLMC(X_train, y_train, V, c, t, X_train.shape[0], k)
        K[count, l] = k
        y_LMC = predictLMC(X_test, V, c, k)
        y_LMC[y_LMC == 0] = 1
        y_pred = y_LMC
        prec[count, l] = precision_score(y_test, y_pred, average="weighted")
        recal[count, l] = recall_score(y_test, y_pred, average="weighted")
        f1[count, l] = f1_score(y_test, y_pred, average="weighted")
        acc[count, l] = accuracy_score(y_test, y_pred)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred) 
        auc[count, l] = metrics.auc(fpr, tpr)
        #print t, count , l, acc[count, l], k
    print t, (K[count, 0]+K[count,1])*1.0/2, (prec[count, 0]+prec[count,1])*1.0/2, (recal[count, 0]+recal[count,1])*1.0/2, (f1[count, 0]+f1[count,1])*1.0/2, (acc[count, 0]+acc[count,1])*1.0/2,(auc[count, 0]+auc[count,1])*1.0/2
    count += 1

1
1 272.0 0.838504507798 0.83851554664 0.838420190152 0.83851554664 0.833111088325
2
2 495.5 0.840536898472 0.841023069208 0.840684313484 0.841023069208 0.832810664203
3
3 741.5 0.839056965233 0.839017051153 0.838469272965 0.839017051153 0.831551850795
4
4 989.0 0.839212363633 0.839518555667 0.838948989434 0.839518555667 0.83137655649
5
5 1135.0 0.832838217124 0.832998996991 0.831785304088 0.832998996991 0.821727359854
10
10 2171.5 0.836328974562 0.836008024072 0.835850200236 0.836008024072 0.831461394614
15
15 3412.0 0.828497075376 0.828986960883 0.828493182909 0.828986960883 0.820930336143
20
20 4651.5 0.844750944912 0.844533600802 0.844406745052 0.844533600802 0.839822334682
25
25 5366.5 0.8334305067 0.833500501505 0.833126062172 0.833500501505 0.825884301561
50
50 11085.5 0.839881487771 0.83851554664 0.838932012941 0.83851554664 0.83669910074
75
75 16591.0 0.833658554114 0.834002006018 0.833772347891 0.834002006018 0.828292367107
100
100 22578.0 0.844511614716 0.844032096289 0.8438